<a href="https://colab.research.google.com/github/TommyChoiDS/ds-section1-project/blob/main/AI_05_%EC%B5%9C%EC%A4%91%ED%9B%88_Section1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data , Library import

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px


In [ ]:
df = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/datasets/vgames2.csv',index_col = 0)

In [ ]:
pd.set_option('display.max_row', None)
pd.set_option('display.max_columns', None)

In [ ]:
df

# EDA

# Data Preprocessing

###  Year Data 전처리

In [ ]:
# Year 데이터중 잘못 표기된 데이터 확인 
# 1800년 이전에는 비디오게임이 없었기 때문에 Year변수의 값이 1800이하인 데이터의 수를 파악
df.loc[df['Year'] < 1800,:].value_counts(df['Year'])

In [ ]:
# 50이상인 데이터는 1950년 ~ 2000년 데이터로 판단.
# 50이하인 데이터는 2000년 이후 데이터로 판단.

def correct_year(value):
  if (value < 1800) & (50 < value):
    value = value + 1900
    return value
  elif value < 50:
    value = value + 2000
    return value
  else:
    return value

In [ ]:
# 연도 데이터 수정
# 연도 데이터는 연도 데이터끼리 연산이 필요하지 않고, 연도별 데이터를 group by하기 위해 문자형 데이터로 변환하는것이 유용하다고 판단.
df['Year'] = df['Year'].apply(correct_year).astype(pd.Int64Dtype()).astype(str)

# correct_year함수로 연도데이터 수정. 
# .astype(Int64Dtype)로 NaN값을 제외한 나머지 연도 데이터를 정수로 변환. 
# .astype(str)로 문자형으로 변환.

### Sales Data 전처리

In [ ]:
# K는 1,000, M은 1,000,000으로 변환.
# K, M으로 표기되지 않은 데이터는 
def correct_scale(value):
  if value[-1] == 'K':
    value = value.replace('K','')
    value = float(value)*0.001
    return value
  elif value[-1] == 'M':
    value = value.replace('M','')
    return value
  else:
    return value

In [ ]:
df['NA_Sales'] = pd.to_numeric(df['NA_Sales'].apply(correct_scale))
df['EU_Sales'] = pd.to_numeric(df['EU_Sales'].apply(correct_scale))
df['JP_Sales'] = pd.to_numeric(df['JP_Sales'].apply(correct_scale))
df['Other_Sales'] = pd.to_numeric(df['Other_Sales'].apply(correct_scale))

### Missing Data 전처리

In [ ]:
# 각 column별 결측치 수
df.isnull().sum(axis = 0)

In [ ]:
# 우선 결측치가 있는 데이터 행을 뽑아보자
df.loc[df.isnull().sum(axis = 1) == True,:]


# 필수사항

###  지역에 따른 장르 선호도 (필수사항)

In [ ]:
# 장르별 누적 판매량
groupby_genre = df.groupby('Genre', as_index = False).sum()
groupby_genre

###  연도별 게임 트렌드 (필수사항)

In [ ]:
groupby_year = df.groupby('Year', as_index = False).sum()
groupby_year

In [ ]:
# 연도 및 장르별 판매량
df_groupby = df.groupby(['Year','Genre'], as_index = False).sum()
df_groupby['Total_Sales'] = df_groupby.iloc[:,2:].sum(axis = 1)
df_groupby

In [ ]:
# 연도별 게임 장르 트렌드 시각화
fig2 = px.line(df_groupby, x = 'Year', y = 'Total_Sales', color = 'Genre',title = 'Game Trend')
fig2.show()

###  출고량이 높은 게임에 대한 분석 및 시각화 프로세스 (필수사항)

In [ ]:
df.dtypes

In [ ]:
# 멀티플랫폼 출시, 리마스터와 같은 이유로 동일한 이름의 중복데이터를 하나의 이름으로 병합.
df_title = df.groupby('Name', as_index = False).sum()
df_title['Total_Sales'] = df_title.iloc[:,1:].sum(axis = 1)
df_title.sort_values(by = 'Total_Sales')

In [ ]:
# fig.write_html("./fig_NA.html")